In [14]:
import pandas as pd
import music_recommend

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#emotion = { "joy":0.2, "excitement":0.5, "gratitude":0.7 }
emotion = { "grief":0.2, "curiosity":0.5, "disapproval":0.7 }

music_recommend.get_music(emotion, None, None, True)

,FILE,ENG,KOR,score,GENRE,TEMPO,MOOD,INSTRUMENT
75,76. Sarabande_ 사라방드_국악.mp3,Sarabande,사라방드,4.1330,국악,조금느리게,SAD,장구(JANGGU)+해금(HAEGEUM)+피아노(PIANO)+대금(DAEGEUM)
82,83. Moonlight_ 월광_국악.mp3,Moonlight,월광,4.1180,국악,조금느리게,SAD,피아노(PIANO)+대금(DAEGEUM)
23,24. Minuet In G Major BWV Anh. II 114_ 미뉴에트 사장...,Minuet In G Major BWV Anh. II 114,미뉴에트 사장조 BWV Anh. II 114,4.0730,funk,조금빠르게,POWERFUL,콘트라베이스(CONTRABASS)+피아노(PIANO)+드럼(DRUM)
78,79. Solveig's Song_ 솔베이지의 노래_국악.mp3,Solveig's Song,솔베이지의 노래,4.0680,국악,보통빠르기로,SAD,피아노(PIANO)+대금(DAEGEUM)+스트링앙상블(STRING ENSEMBLE)
95,"96. Invention No.4 in D minor, BWV 775_ 인벤션 4번...","Invention No.4 in D minor, BWV 775",인벤션 4번 라단조,3.9920,string,보통빠르기로,SAD,관악기(BRASS&WOODWIND)+스트링(STRING)
70,71. Consolation_ 위안_uptempo swing.mp3,Consolation,위안,3.9830,uptempo swing,매우빠르게,POWERFUL,콘트라베이스(CONTRABASS)+피아노(PIANO)+드럼(DRUM)
61,62. blumenlied (Flower Song)_ 꽃노래_uptempo swin...,blumenlied (Flower Song),꽃노래,3.9830,uptempo swing,매우빠르게,POWERFUL,콘트라베이스(CONTRABASS)+피아노(PIANO)+드럼(DRUM)
28,29. Salut D'amour In E Major Op.12_ 사랑의 인사 마장조...,Salut D'amour In E Major Op.12,사랑의 인사 마장조 작품번호 12,3.9830,uptempo swing,매우빠르게,POWERFUL,콘트라베이스(CONTRABASS)+피아노(PIANO)+드럼(DRUM)
16,17. Nocturne Op.9-2_ 녹턴 작품번호 9-2_funk.mp3,Nocturne Op.9-2,녹턴 작품번호 9-2,3.9830,funk,느리게,WHITE,콘트라베이스(CONTRABASS)+피아노(PIANO)+기타(GUITAR)+드럼(DRUM)
45,46. Solveing's song_ 솔베이그의 노래_swing.mp3,Solveing's song,솔베이그의 노래,3.9680,swing,보통빠르기로,SAD,콘트라베이스(CONTRABASS)+피아노(PIANO)+드럼(DRUM)
